### prove of concept (POC)

In [1]:
# Import libraries
import os
import sys

current_dir = os.getcwd()
sys.path.append(os.path.join(current_dir,".."))

from utils.helper import set_openai_key, test_openai_api, create_openai_client, count_tiktoken_length

print("Import successfully!")

Import successfully!


In [2]:
# Intialize
set_openai_key() # Set openai key
test_openai_api() # Test openai api
client = create_openai_client() # Create openai client

API key set successfully.
This is a test.


In [3]:
# Create chunk of PDF
!python ../utils/extract_pdf_by_pages.py ../assets/Fortran77_tutorial.pdf tmp.pdf 2 3

Extracted pages 2 to 3 into 'tmp.pdf'


In [4]:
# Convert to markdown file
!python ../utils/export_pdf_to_markdown.py tmp.pdf tmp.txt

Fetching 9 files: 100%|█████████████████████████| 9/9 [00:00<00:00, 7023.02it/s]
Markdown content exported successfully to: tmp.txt


In [11]:
# Create question
with open("tmp.txt","r",encoding="utf-8") as f:
    content = f.read()

# messages = [
#     {"role":"system", "content":"You are an expert in training large language models."},
#     {"role":"user", "content":"Below is the content that needs to be used for generating a question."},
#     {"role":"user", "content":f"{content}"},
#     {"role":"user", "content":"Based on the above content, generate a clear and detailed question that would allow someone to answer it without referring back to the content. The question should contain enough information and context for answering. If the question base on an example code, make sure the question retains that example code in the question itself. Your response should only be the question, without any extra information or explanation."},
# ]

messages = [
    {"role": "system", "content": "You are an expert in training large language models."},
    {"role": "user", "content": "Below is the content for generating a question:"},
    {"role": "user", "content": content},
    {"role": "user", "content": (
        "Based on the above content, generate a clear and detailed question that would allow someone to answer it without referring back to the content"
        "The question should contain enough information and context for answering."
        "If the question base on an example code, make sure the question retains that example code in the question itself."
        "Your response should only be the question, without any extra information or explanation."
    )}
]


tiktoken_length = count_tiktoken_length(messages)
print(tiktoken_length)

1272


In [12]:
response = client.chat.completions.create(
    messages=messages,
    model="gpt-4o",
)

question = response.choices[0].message.content
print(question)

What are the key formatting rules and conventions of Fortran 77, including the column position rules, and how do they apply to writing and organizing a basic Fortran program, such as the example below that calculates the area of a circle?

```
program circle
    real r, area
    c This program reads a real number r and prints
    c the area of a circle with radius r.
    write (*,*) 'Give radius r:'
    read (*,*) r
    area = 3.14159*r*r
    write (*,*) 'Area = ', area
stop
end
```

Discuss the function of comments, the role of column positions in organizing code, and the significance of the `stop` statement in Fortran 77.


In [13]:
# Create answer
messages = [
    {"role":"system", "content":"You programming expert"},
    {"role":"user", "content":"Please answer the following question"},
    {"role":"user", "content":f"{question}"},
]

tiktoken_length = count_tiktoken_length(messages)
print(tiktoken_length)

168


In [14]:
response = client.chat.completions.create(
    messages=messages,
    model="gpt-4o",
)

answer = response.choices[0].message.content
print(answer)

Fortran 77, being an older programming language, comes with a set of rules and conventions that stem from its history on punch card systems. Here's an overview of key formatting rules and conventions:

### Column Positions in Fortran 77

1. **Columns 1-5: Statement Labels**
   - These are optional and used for numeric labels that serve as references for `GOTO` or other control flow statements. Each label is unique and usually placed in one of these columns.
   - Example: `10 CONTINUE`

2. **Column 6: Continuation Character**
   - A non-blank character (commonly a character like a plus `+` or any non-numeric character) in column 6 indicates that the line is a continuation of the previous line.
   - This is used for splitting long statements over multiple lines for clarity.

3. **Columns 7-72: Statements**
   - The actual Fortran statements should be written in these columns.
   - Statements that extend beyond column 72 are not considered by the compiler.

4. **Columns 73-80: Comments or

In [15]:
# Create instruction data
instruct_data = [
    {"role":"system", "content":"You programming expert"},
    {"role":"user", "content":"Please answer the following question"},
    {"role":"user", "content":f"{question}"},
    {"role":"user", "content":f"{answer}"},
]

print(instruct_data)

[{'role': 'system', 'content': 'You programming expert'}, {'role': 'user', 'content': 'Please answer the following question'}, {'role': 'user', 'content': "What are the key formatting rules and conventions of Fortran 77, including the column position rules, and how do they apply to writing and organizing a basic Fortran program, such as the example below that calculates the area of a circle?\n\n```\nprogram circle\n    real r, area\n    c This program reads a real number r and prints\n    c the area of a circle with radius r.\n    write (*,*) 'Give radius r:'\n    read (*,*) r\n    area = 3.14159*r*r\n    write (*,*) 'Area = ', area\nstop\nend\n```\n\nDiscuss the function of comments, the role of column positions in organizing code, and the significance of the `stop` statement in Fortran 77."}, {'role': 'user', 'content': 'Fortran 77, being an older programming language, comes with a set of rules and conventions that stem from its history on punch card systems. Here\'s an overview of k

### random sampling
- temperature
- top_k
- top_n
- top_p
- token_length

In [24]:
temperature=0.5 #0.7
top_n=5
top_p=0.8
token_length=256

response = client.chat.completions.create(
    model="gpt-4",  # Specify the model you want to use
    messages=messages,
    temperature=temperature,
    top_p=top_p,
    n=top_n,
)

answer = response.choices[0].message.content
print(answer)

Fortran 77 is a high-level programming language designed for numerical computations. It has specific formatting rules and conventions, including column position rules, that must be followed when writing and organizing a Fortran program.

Column Position Rules:
1. Columns 1-5: These columns are for statement labels, which are numerical and used for `GO TO` statements and format identifiers. The example provided does not use any statement labels.
2. Column 6: This column is for continuation of the previous line. If a statement does not fit within a single line (72 columns), it can be continued on the next line by placing any non-space character in the sixth column of the continued line.
3. Columns 7-72: These columns are for statements. All Fortran 77 statements must start in these columns.
4. Columns 73-80: These columns are often used for sequence numbers and are ignored by the compiler.

Comments:
In Fortran 77, comments are denoted by the character 'c' or '*' in the first column. The

### class pipeline